# Assignment Part1

In [1]:
!conda install beautifulsoup4

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0

beautifulsoup4 100% |################################| Time: 0:00:00  40.73 MB/s


In [2]:
!conda install lxml

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libgcc-ng: 7.2.0-h7cc24e2_2     --> 8.2.0-hdf63c60_1    
    libxml2:   2.9.4-h6b072ca_5     --> 2.9.8-hf84eae3_0    
    libxslt:   1.1.29-hcf9102b_5    --> 1.1.32-h1312cb7_0   
    lxml:      4.1.0-py35ha401a81_0 --> 4.2.5-py35hefd8a0e_0

libgcc-ng-8.2. 100% |################################| Time: 0:00:00  53.96 MB/s
libxml2-2.9.8- 100% |################################| Time: 0:00:00  66.90 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  22.26 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  64.64 MB/s


In [3]:
!conda install requests

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    requests: 2.18.4-py35hb9e6ad1_1 --> 2.19.1-py35_0

requests-2.19. 100% |################################| Time: 0:00:00  35.96 MB/s


In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source,'lxml')

In [4]:
table = soup.find_all('table')[0]
#table

In [6]:
#print(soup.prettify())

In [5]:
table_raw = soup.find('table', 'tr',class_='wikitable sortable')

In [8]:
#print(table_raw.prettify())

In [6]:
b = table_raw.tbody.text
#b

In [7]:
c = b.split('\n\n\n')
#c

In [8]:
d = pd.DataFrame(c)
#d

In [9]:
new = d[0].str.split('\n',n=0,expand = True)

new.drop(columns =[3,4], inplace = True) 
new.drop([0],inplace=True)
new.columns=['PostalCode','Borough','Neighborhood']
new = new.reset_index(drop=True)
new.shape
new.tail()

,PostalCode,Borough,Neighborhood
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


In [10]:
new.shape

(289, 3)

In [11]:
for i in range (0,(len(new['Borough']))):
    
    if(new.iloc[i,1] == 'Not assigned'):
        new.iloc[i,1]=None
    else:
        pass
print(new.head())

  PostalCode           Borough      Neighborhood
0        M1A              None      Not assigned
1        M2A              None      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront


In [12]:
new = new.dropna(subset=["Borough"],axis=0)
new = new.reset_index(drop=True)
new.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
for i in range (0,(len(new['Neighborhood']))):
    if(new.iloc[i,2] == 'Not assigned'):
        new.iloc[i,2]=new.iloc[i,1]
    else:
        pass
new.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [14]:
group1 = new["Neighborhood"].groupby(new["PostalCode"]).agg([('Neighborhood', ', '.join)])
group1=group1.reset_index()
print(group1.size)


206


In [15]:
df_inner = pd.merge(new, group1, on = "PostalCode", how ='inner')
df_inner.head()

,PostalCode,Borough,Neighborhood_x,Neighborhood_y
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park,"Harbourfront, Regent Park"
4,M6A,North York,Lawrence Heights,"Lawrence Heights, Lawrence Manor"


In [16]:
df_inner.drop(columns=["Neighborhood_x"], inplace=True)


In [17]:
df_inner.shape

(212, 3)

In [18]:
df_inner = df_inner.drop_duplicates()

In [19]:
df_inner.columns=['PostalCode','Borough','Neighborhood']
df_inner = df_inner.reset_index(drop=True)
df_inner.shape

(103, 3)

In [21]:
df_inner.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [22]:
#Number of rows in the final dataframe
df_inner.shape

(103, 3)

# Assignment Part2

In [23]:
geo_cord = pd.read_csv('https://cocl.us/Geospatial_data')

In [24]:
geo_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
geo_cord.columns=["PostalCode","Latitude","Longitude"]    

In [26]:
geo_df = pd.merge(df_inner,geo_cord, on="PostalCode", how="inner")
geo_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Assignment Part3

### Segmenting and Clustering Neighborhoods in Toronto

In [27]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.17 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  25.11 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  27.23 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  35.15 MB/s
Libraries imported.


In [28]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.03 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.61 MB/s


#### Use geopy library to get the latitude and longitude values of Toronto.

In [29]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [30]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(geo_df['Latitude'], geo_df['Longitude'], geo_df['Borough'], geo_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Downtown Toronto.

In [31]:
downtown_data = geo_df[geo_df["Borough"] == "Downtown Toronto"].reset_index(drop=True)
downtown_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


Let's get the geographical coordinates of Downtown Toronto.

In [32]:
address = "Downtown Toronto, Ontario"
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


Let's visualize Downtown Toronto and the neighborhoods in it.

In [33]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude,longitude], zoom_start=11)
# add markers to map
for lat, lng, label in zip(downtown_data["Latitude"], downtown_data["Longitude"], downtown_data["Neighborhood"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color="green",
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_downtown)           

map_downtown


Define Foursquare Credentials and Version:

In [89]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the fourth neighborhood in our dataframe.

In [35]:
#Get the neighborhood's name.
downtown_data.loc[4,'Neighborhood']

'Central Bay Street'

Get the neighborhood's latitude and longitude values.

In [36]:
neighborhood_latitude = downtown_data.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [37]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [38]:
#RESULTS
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf455dddd57972380abf399'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-537d4d6d498ec171ba22e7fe-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '537d4d6d498ec171ba22e7fe',
       'location': {'address': '82 Gerrard Street W',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Gerrard & LaPlante',
        'distance': 151,
        'formattedAddress': ['82 Gerrard Street W (Gerrard & LaPlante)',
         'Toronto ON M5G 1Z4',
         'Canada'],
   

To clean the json and structure it into a *pandas* dataframe.

In [39]:
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-537d4d6d498ec171ba22e7fe-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'primary': True,
     'shortName': 'Coffee Shop'}],
   'id': '537d4d6d498ec171ba22e7fe',
   'location': {'address': '82 Gerrard Street W',
    'cc': 'CA',
    'city': 'Toronto',
    'country': 'Canada',
    'crossStreet': 'Gerrard & LaPlante',
    'distance': 151,
    'formattedAddress': ['82 Gerrard Street W (Gerrard & LaPlante)',
     'Toronto ON M5G 1Z4',
     'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65842123574496,
      'lng': -79.38561319551111}],
    'lat': 43.65842123574496,
    'lng': -79.38561319551111,
    'postalCode

In [40]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [41]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head(2)

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'type': 'general', 'reasonName': 'globalInte...",e-0-537d4d6d498ec171ba22e7fe-0,"[{'id': '4bf58dd8d48988d1e0931735', 'primary':...",537d4d6d498ec171ba22e7fe,82 Gerrard Street W,CA,Toronto,Canada,Gerrard & LaPlante,...,"[{'label': 'display', 'lat': 43.65842123574496...",43.658421,-79.385613,NaN,M5G 1Z4,ON,Jimmy's Coffee,0,[],NaN
1,0,"[{'type': 'general', 'reasonName': 'globalInte...",e-0-557231e3498e540f05f3083c-1,"[{'id': '4bf58dd8d48988d1e0931735', 'primary':...",557231e3498e540f05f3083c,70 Gerrard St West,CA,Toronto,Canada,Bay St,...,"[{'label': 'display', 'lat': 43.65856999999997...",43.658570,-79.385123,NaN,M5G 1J5,ON,Tim Hortons,0,[],NaN


In [42]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head(2)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Jimmy's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'primary':...",43.658421,-79.385613
1,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'primary':...",43.658570,-79.385123


In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues['venue.categories'].head(2)

0    Coffee Shop
1    Coffee Shop
Name: venue.categories, dtype: object

In [45]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(2)

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123


And how many venues were returned by Foursquare?

In [46]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

86 venues were returned by Foursquare.


#### Explore Neighborhoods in Downtown Toronto

Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called *DowntownToronto_venues*.

In [48]:
DowntownToronto_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


Let's check the size of the resulting dataframe

In [49]:
print(DowntownToronto_venues.shape)
DowntownToronto_venues.head()

(1283, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [50]:
DowntownToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [51]:
print('There are {} uniques categories.'.format(len(DowntownToronto_venues['Venue Category'].unique())))

There are 209 uniques categories.


#### Next, let's group rows by neighborhood, by taking the mean of the frequency of occurrence of each category

In [55]:
DowntownToronto_grouped = DowntownToronto_onehot.groupby('Neighborhood').mean().reset_index()
DowntownToronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.010000,0.0,0.01
1,Berczy Park,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00
3,"Cabbagetown, St. James Town",0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00
4,Central Bay Street,0.011628,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.011628,0.0,0.0,0.0,0.011628,0.0,0.00


Let's print each neighborhood along with the top 5 most common venues

In [56]:
num_top_venues = 5

for hood in DowntownToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DowntownToronto_grouped[DowntownToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2    Restaurant  0.06
3        Bakery  0.04
4   Cheese Shop  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1  Airport Terminal  0.14
2   Airport Service  0.14
3             Plane  0.07
4  Sculpture Garden  0.07


----Cabbagetown, St. James Town----
               venue  freq
0        Coffee Shop  0.09
1         Restaurant  0.09
2               Café  0.06
3             Bakery  0.04
4  Indian Restaurant  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.06
2                Café  0.06
3        Burger Joint  0.

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DowntownToronto_grouped['Neighborhood']

for ind in np.arange(DowntownToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DowntownToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Gym,Breakfast Spot,Hotel,Cosmetics Shop
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Café
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Park,Indian Restaurant,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Bar,Chinese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Dessert Shop
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Baby Store
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Burger Joint,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Pub,Fast Food Restaurant
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Steakhouse,Gastropub,Seafood Restaurant
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Italian Restaurant,Sports Bar,Gastropub,Deli / Bodega


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [75]:
# set number of clusters
kclusters = 5

DowntownToronto_grouped_clustering = DowntownToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DowntownToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 3, 0, 4, 0, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [80]:
downtown_data.drop(columns="PostalCode",inplace=True)

In [81]:
DowntownToronto_merged = downtown_data

# add clustering labels
DowntownToronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DowntownToronto_merged = DowntownToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DowntownToronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Park,Restaurant,Breakfast Spot,Pub,Theater,Mexican Restaurant,Dessert Shop
1,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room,Bar,Ramen Restaurant
2,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Hotel,Cocktail Bar,Clothing Store,Cosmetics Shop,Gastropub,Bakery,Seafood Restaurant
3,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Pub,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Café
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant


Finally, let's visualize the resulting clusters

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DowntownToronto_merged['Latitude'], DowntownToronto_merged['Longitude'], DowntownToronto_merged['Neighborhood'], DowntownToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster1

In [83]:
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 0, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront, Regent Park",Coffee Shop,Bakery,Café,Park,Restaurant,Breakfast Spot,Pub,Theater,Mexican Restaurant,Dessert Shop
1,"Ryerson, Garden District",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room,Bar,Ramen Restaurant
3,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Café
5,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Baby Store
7,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Scenic Lookout,Sports Bar,Italian Restaurant,Brewery,History Museum
11,"Chinatown, Grange Park, Kensington Market",Café,Bar,Chinese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Dessert Shop
15,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Park,Indian Restaurant,Japanese Restaurant
16,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Seafood Restaurant,Gastropub,Deli / Bodega,Bakery
17,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Burger Joint,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Pub,Fast Food Restaurant


#### Cluster2

In [84]:
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 1, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Restaurant,Café,Hotel,Cocktail Bar,Seafood Restaurant,Pub,Italian Restaurant,Beer Bar,Cheese Shop


#### Cluster3

In [85]:
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 2, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Cocktail Bar,Clothing Store,Cosmetics Shop,Gastropub,Bakery,Seafood Restaurant


#### Cluster4

In [86]:
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 3, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant
8,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Italian Restaurant,Sports Bar,Gastropub,Deli / Bodega
9,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Steakhouse,Gastropub,Seafood Restaurant
10,"Harbord, University of Toronto",Café,Bookstore,Bar,Japanese Restaurant,Coffee Shop,Bakery,Restaurant,Beer Bar,Comfort Food Restaurant,Beer Store
12,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport
13,Rosedale,Park,Playground,Trail,Women's Store,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster5

In [87]:
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 4, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Gym,Breakfast Spot,Hotel,Cosmetics Shop
